In [1]:
%pylab inline
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [2]:
import tensorflow as tf
from tensorflow.models.rnn import rnn, rnn_cell
import numpy as np
import numpy.random as rng

In [3]:
import pandas.io.data as web
from pykalman import KalmanFilter
import numpy as np
import pandas as pd

In [4]:
def get_prices(symbol):
    start, end = '2007-05-02', '2016-04-11'
    data = web.DataReader(symbol, 'yahoo', start, end)
    data=pd.DataFrame(data)
    prices=data['Adj Close']
    #prices=np.asarray(list(prices))
    prices=prices.astype(float)
    return prices

gs = get_prices('GS')

def get_returns(prices):
    return (prices-prices.shift(-1))/prices

rets = get_returns(gs)

def sort_data(rets, N):
    ins = []
    outs = []
    for i in range(N):
        xx = np.random.randint(len(rets)-100)
        ins.append(rets[xx:xx+100].tolist())
        outs.append(rets[xx+100])
    return np.array(ins), np.array(outs)

ins, outs = sort_data(rets, 1000)



In [5]:
xdata = np.array(rets[:-1])
ydata = np.copy(xdata)
ydata[:-1] = xdata[1:]
ydata[-1] = xdata[0]

In [6]:
sess = tf.InteractiveSession()

In [7]:
x = tf.placeholder(tf.float32, [None, 1])
y_ = tf.placeholder(tf.float32, [None, 1])

In [8]:
learning_rate = 0.001
#training_iters = 1000
#batch_size = 128
#display_step = 10

# Network Parameters
#n_input = 1 # MNIST data input (img shape: 28*28)
#n_steps = 100 # timesteps
#n_hidden = 100 # hidden layer num of features
#n_classes = 1 # MNIST total classes (0-9 digits)

# tf Graph input


# Define weights
weights = {
    'hidden': tf.Variable(tf.random_normal([1, 100])),
    'out': tf.Variable(tf.random_normal([100, 1]))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([100])),
    'out': tf.Variable(tf.random_normal([1]))
}


In [9]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_hidden)
    
    # Permuting batch_size and n_steps
    #x = tf.transpose(x)
    # Reshaping to (n_steps*batch_size, n_input)
    #x = tf.reshape(x, [-1, n_input])
    
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_hidden)
    x = tf.split(0, len(xdata), x)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn_cell.BasicLSTMCell(100, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']



In [10]:
pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.l2_loss(pred - y_))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


# Initializing the variables
init = tf.initialize_all_variables()

In [11]:
sess.run(init)

for epoch in range(2000):
    #batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(optimizer, feed_dict={x: xdata.reshape(1,-1).T.shape, y_: ydata})
    if (epoch+1)%100== 0:
        c = sess.run(cost, feed_dict={x: xdata, y_: ydata})
        print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c), \
            )

ValueError: Cannot feed value of shape (2,) for Tensor 'Placeholder:0', which has shape (Dimension(None), Dimension(1))

In [19]:
xdata.reshape(1,-1).T.shape

(2251, 1)

In [7]:
learning_rate = 0.001
training_iters = 1000
#batch_size = 128
#display_step = 10

# Network Parameters
n_input = 1 # MNIST data input (img shape: 28*28)
n_steps = 100 # timesteps
n_hidden = 100 # hidden layer num of features
n_classes = 1 # MNIST total classes (0-9 digits)

# tf Graph input


# Define weights
weights = {
    'hidden': tf.Variable(tf.random_normal([n_input, n_hidden])),
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}


In [8]:
def RNN(x, weights, biases):

    # Prepare data shape to match `rnn` function requirements
    # Current data input shape: (batch_size, n_steps, n_input)
    # Required shape: 'n_steps' tensors list of shape (batch_size, n_hidden)
    
    # Permuting batch_size and n_steps
    x = tf.transpose(x)
    # Reshaping to (n_steps*batch_size, n_input)
    #x = tf.reshape(x, [-1, n_input])
    
    # Split to get a list of 'n_steps' tensors of shape (batch_size, n_hidden)
    x = tf.split(0, n_steps, x)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn_cell.BasicLSTMCell(n_hidden, forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    return tf.matmul(outputs[-1], weights['out']) + biases['out']

pred = RNN(x, weights, biases)

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.l2_loss(pred - y_))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


# Initializing the variables
init = tf.initialize_all_variables()

ValueError: Linear expects shape[1] of arguments: [[1, None], [None, 100]]

(100, 1000)

In [9]:
sess.run(init)

for epoch in range(2000):
    #batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(optimizer, feed_dict={x: ins, y_: outs.reshape(1,-1).T})
    if (epoch+1)%100== 0:
        c = sess.run(cost, feed_dict={x: ins, y_: outs.reshape(1,-1).T})
        print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(c), \
            )

InvalidArgumentError: Incompatible shapes: [10,1] vs. [1000,1]
	 [[Node: gradients/sub_grad/BroadcastGradientArgs = BroadcastGradientArgs[_device="/job:localhost/replica:0/task:0/cpu:0"](gradients/sub_grad/Shape, gradients/sub_grad/Shape_1)]]
Caused by op 'gradients/sub_grad/BroadcastGradientArgs', defined at:
  File "/Users/liamconnell/anaconda/lib/python3.4/runpy.py", line 170, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/liamconnell/anaconda/lib/python3.4/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/config/application.py", line 574, in launch_instance
    app.start()
  File "/Users/liamconnell/anaconda/lib/python3.4/site-packages/IPython/kernel/zmq/kernelapp.py", line 373, in start
    ioloop.IOLoop.instance().start()
  File "/Users/liam

In [ ]:
sess.run(init)
step = 1
# Keep training until reach max iterations
while step * batch_size < training_iters:
    #batch_x, batch_y = mnist.train.next_batch(batch_size)
    # Reshape data to get 28 seq of 28 elements
    batch_x = batch_x.reshape((batch_size, n_steps, n_input))
    # Run optimization op (backprop)
    sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})
    if step % display_step == 0:
        # Calculate batch accuracy
        acc = sess.run(accuracy, feed_dict={x: batch_x, y: batch_y})
        # Calculate batch loss
        loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y})
        print "Iter " + str(step*batch_size) + ", Minibatch Loss= " + \
              "{:.6f}".format(loss) + ", Training Accuracy= " + \
              "{:.5f}".format(acc)
    step += 1
print "Optimization Finished!"

# Calculate accuracy for 128 mnist test images
test_len = 128
test_data = mnist.test.images[:test_len].reshape((-1, n_steps, n_input))
test_label = mnist.test.labels[:test_len]
print "Testing Accuracy:", \
    sess.run(accuracy, feed_dict={x: test_data, y: test_label})